In [1]:
%matplotlib nbagg
import numpy as np
import matplotlib.pyplot as plt
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
import pickle

import tensorflow.keras.backend as K
from model import TBPP384
from tbpp_prior import PriorUtil
from tbpp_data import InputGenerator
from tbpp_training import TBPPFocalLoss
from utils.model import load_weights
from utils.training import Logger

### Check using devices

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

### Data

In [2]:
from data_cocotext import GTUtility
gt_util = GTUtility('data/COCO-Text', polygon=True)

gt_util_train, gt_util_val = gt_util.split(0.9)

### Data check

In [ ]:
idx = 4
_, image, label = gt_util_train.sample()

In [ ]:
plt.imshow(image)
gt_util_train.plot_gt(label,show_labels=False)
plt.show()

In [ ]:
image.shape

In [ ]:
label[:,6] * 600

In [ ]:
plt.imshow(image)
plt.show()

### Model

In [3]:
# TextBoxes++
K.clear_session()
model = TBPP384(softmax=False)
weights_path = './checkpoints/201911120606_tbpp384fl_synthtext/weights.010.h5'

experiment = 'tbpp384fl_COCOtext'

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
load_weights(model, weights_path)

In [5]:
prior_util = PriorUtil(model)

### Saving parameters

In [ ]:
import json

* Add custom Layer

In [ ]:
from tensorflow.keras.utils import get_custom_objects
from utils.layers import Normalize

In [ ]:
get_custom_objects().update({
    "Normalize": Normalize
})

In [ ]:
with open("../model_params.json",'w') as f:
    f.write(model.to_json())

In [ ]:
from tensorflow.keras.models import model_from_json

In [ ]:
with open("../model_params.json",'r') as f:
    parameter_saved_model = model_from_json(f.read())

### Training

In [ ]:
# os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

* checking available batch size of validation set

```
gen_val = InputGenerator(
    gt_util_val, prior_util, batch_size*4, model.image_size)
```python

In [6]:
from multiprocessing import cpu_count
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
a, b = next(gen_train.generate())

In [ ]:
a.shape

In [ ]:
model.train_on_batch(a,b)

In [ ]:
gen_train.num_batches/4

In [9]:
epochs = 200
batch_size = 32

gen_train = InputGenerator(gt_util_train, prior_util, batch_size, model.image_size)
gen_val = InputGenerator(gt_util_val, prior_util, batch_size*4, model.image_size)

checkdir = './checkpoints/' + time.strftime('%Y%m%d%H%M') + '_' + experiment
if not os.path.exists(checkdir):
    os.makedirs(checkdir)

with open(checkdir+'/source.py','wb') as f:
    source = ''.join(['# In[%i]\n%s\n\n' % (i, In[i]) for i in range(len(In))])
    f.write(source.encode())

# optim = tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9, decay=0, nesterov=True)
optim = tf.keras.optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=0.001, decay=0.0)

# weight decay
regularizer = tf.keras.regularizers.l2(5e-4) # None if disabled
#regularizer = None
for l in model.layers:
    if l.__class__.__name__.startswith('Conv'):
        l.kernel_regularizer = regularizer

loss = TBPPFocalLoss(lambda_conf=10000.0, lambda_offsets=1.0)

model.compile(optimizer=optim, loss=loss.compute, metrics=loss.metrics)

callbacks = [
    ModelCheckpoint(checkdir+'/weights.{epoch:03d}.h5', 
                    verbose=1, save_weights_only=True),
    Logger(checkdir)
]

print(checkdir.split('/')[-1])
history = model.fit_generator(
        gen_train.generate(),
        epochs=epochs, 
        steps_per_epoch=int(gen_train.num_batches/4), 
        callbacks=callbacks,
        validation_data=gen_val.generate(), 
        validation_steps=int(gen_val.num_batches/4),
        workers=cpu_count(), 
        use_multiprocessing=True)

201911160931_tbpp384fl_COCOtext
Epoch 1/200
102/103 [============================>.] - ETA: 0s - loss: 23.1156WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00001: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.001.h5
103/103 [==============================] - 98s 955ms/step - loss: 23.1024 - val_loss: 26.8745
Epoch 2/200
102/103 [============================>.] - ETA: 0s - loss: 21.9998WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00002: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.002.h5
103/103 [==============================] - 83s 809ms/step - loss: 21.9840 - val_loss: 25.5745
Epoch 3/200
102/103 [============================>.] - ETA: 0s - loss: 20.5324WARNING:tensorflow:Usi


Epoch 00019: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.019.h5
103/103 [==============================] - 87s 842ms/step - loss: 18.0322 - val_loss: 21.8399
Epoch 20/200
102/103 [============================>.] - ETA: 0s - loss: 15.8321WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00020: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.020.h5
103/103 [==============================] - 81s 786ms/step - loss: 15.8021 - val_loss: 21.8461
Epoch 21/200
102/103 [============================>.] - ETA: 0s - loss: 15.7026WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00021: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.021.h5
103/103 [==============================


Epoch 00038: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.038.h5
103/103 [==============================] - 80s 779ms/step - loss: 14.7819 - val_loss: 20.9119
Epoch 39/200
102/103 [============================>.] - ETA: 0s - loss: 14.5716WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00039: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.039.h5
103/103 [==============================] - 85s 823ms/step - loss: 14.5520 - val_loss: 20.7251
Epoch 40/200
102/103 [============================>.] - ETA: 0s - loss: 14.2582WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00040: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.040.h5
103/103 [==============================


Epoch 00057: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.057.h5
103/103 [==============================] - 86s 836ms/step - loss: 14.6971 - val_loss: 20.8553
Epoch 58/200
102/103 [============================>.] - ETA: 0s - loss: 14.2051WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00058: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.058.h5
103/103 [==============================] - 82s 791ms/step - loss: 14.2404 - val_loss: 20.8038
Epoch 59/200
102/103 [============================>.] - ETA: 0s - loss: 14.9278WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00059: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.059.h5
103/103 [==============================


Epoch 00076: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.076.h5
103/103 [==============================] - 81s 787ms/step - loss: 13.2401 - val_loss: 20.6761
Epoch 77/200
102/103 [============================>.] - ETA: 0s - loss: 15.2617WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00077: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.077.h5
103/103 [==============================] - 85s 821ms/step - loss: 15.2408 - val_loss: 20.9236
Epoch 78/200
102/103 [============================>.] - ETA: 0s - loss: 13.2567WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00078: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.078.h5
103/103 [==============================

...


Epoch 00114: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.114.h5
103/103 [==============================] - 85s 826ms/step - loss: 14.3158 - val_loss: 20.8006
Epoch 115/200
102/103 [============================>.] - ETA: 0s - loss: 12.7453WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00115: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.115.h5
103/103 [==============================] - 88s 857ms/step - loss: 12.7861 - val_loss: 21.0046
Epoch 116/200
102/103 [============================>.] - ETA: 0s - loss: 10.9823WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00116: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.116.h5
103/103 [============================


Epoch 00133: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.133.h5
103/103 [==============================] - 81s 788ms/step - loss: 12.8297 - val_loss: 21.2975
Epoch 134/200
102/103 [============================>.] - ETA: 0s - loss: 12.3838WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00134: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.134.h5
103/103 [==============================] - 85s 830ms/step - loss: 12.3877 - val_loss: 21.1365
Epoch 135/200
102/103 [============================>.] - ETA: 0s - loss: 12.1715WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00135: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.135.h5
103/103 [============================


Epoch 00152: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.152.h5
103/103 [==============================] - 85s 828ms/step - loss: 12.8654 - val_loss: 21.3052
Epoch 153/200
102/103 [============================>.] - ETA: 0s - loss: 11.3888WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00153: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.153.h5
103/103 [==============================] - 82s 792ms/step - loss: 11.3867 - val_loss: 21.4674
Epoch 154/200
102/103 [============================>.] - ETA: 0s - loss: 10.4760WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00154: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.154.h5
103/103 [============================


Epoch 00171: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.171.h5
103/103 [==============================] - 87s 845ms/step - loss: 12.1481 - val_loss: 21.5503
Epoch 172/200
102/103 [============================>.] - ETA: 0s - loss: 12.8046WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00172: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.172.h5
103/103 [==============================] - 83s 804ms/step - loss: 12.8861 - val_loss: 21.6295
Epoch 173/200
102/103 [============================>.] - ETA: 0s - loss: 10.6872WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00173: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.173.h5
103/103 [============================


Epoch 00190: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.190.h5
103/103 [==============================] - 85s 825ms/step - loss: 10.4431 - val_loss: 21.5613
Epoch 191/200
102/103 [============================>.] - ETA: 0s - loss: 10.6711WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00191: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.191.h5
103/103 [==============================] - 86s 839ms/step - loss: 10.6535 - val_loss: 21.4723
Epoch 192/200
102/103 [============================>.] - ETA: 0s - loss: 10.6327WARNING:tensorflow:Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence` class.

Epoch 00192: saving model to ./checkpoints/201911160931_tbpp384fl_COCOtext/weights.192.h5
103/103 [============================

### Parameters 

In [ ]:
from utils.model import count_parameters
count_parameters(model)
calc_memory_usage(model)

In [ ]:
# frequency of class instance in local ground truth, used for weightning the focal loss
s = np.zeros(gt_util.num_classes)
for i in range(1000):#range(gt_util.num_samples):
    egt = prior_util.encode(gt_util.data[i])
    s += np.sum(egt[:,-gt_util.num_classes:], axis=0)
sn = np.asarray(np.sum(s))/s
print(np.array(sn, dtype=np.int32))
print(sn/np.sum(sn))